First I need to get my data 

In [1]:
import os
import pandas as pd
import numpy as np
import mypytable as mypy
from sklearn.model_selection import train_test_split
import tensorflow as tf
import copy


2023-04-29 16:00:28.066799: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-04-29 16:00:28.470575: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-04-29 16:00:29.552187: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: :/home/cbarker4/miniconda3/envs/tf/lib/
2023-04-29 16:00:29.552240: W tensorflow/compiler/xla/stre

In [2]:
stockfiles= os.listdir("/home/cbarker4/Documents/DataScience/StockTrader/Data")
# print(stockfiles)

In [3]:
X = []
Y=[]
Xlast=[]
stockfiles=["AAPL.csv"]
for j,val in enumerate(stockfiles):
    print(val)
    if j == 30:
        break

    mt = mypy.MyPyTable()
    mt.load_from_file("/home/cbarker4/Documents/DataScience/StockTrader/Data/"+val)
    # mt.drop_column('v')
    # mt.drop_column('t')
    # mt.drop_column('s')
    # mt.drop_column('h')
    # mt.drop_column('l')
    # mt.drop_column('o')

    mt.drop_column(0)
    i = 100
    while i <  len(mt.data)-1:
    

        table = mt.create_sub_table(i-100,i)
        # big = table.max()
        # little = table.min()
        # table = table.normalize()
        

      
        # print(table.data[0])
        X.append(copy.deepcopy(table.get_column(0)))
        # Xlast.append(table.data[-1])
        # num = int((((mt.get_row(i+1)[0])-little))/(big-little)*1000)/10
        num = mt.get_row(i+1)[0]
        Y.append(num)
        i+=1
    


AAPL.csv


In [4]:
X = np.array(X)
# print(Y)
Y = np.array(Y)
# print(Y.shape)
count = 0
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.25 ,random_state=10)
i=0
for val in X_train:
    # print(val[-1][0])
    # if val[-1][0]<Y_train[i]:
    if val[-1]<Y_train[i]:
        count+=1
    i+=1

print(X_train[-1][-1])
print(Y_train[-1])
print(count/len(Y_train))



149.84
138.2
0.4690265486725664


In [5]:
# Check shapes
print('X_train shape: ', X_train.shape)
print('y_train shape: ', Y_train.shape)
print('X_test shape: ', X_test.shape)
print('y_test shape: ', Y_test.shape)

X_train shape:  (113, 100)
y_train shape:  (113,)
X_test shape:  (38, 100)
y_test shape:  (38,)


In [6]:
def accuracy(y_true,y_pred):

    good = abs(y_true-y_pred)
    if good>.25:
        False
    else:
        True
    


# Build epoch checkpoint callback
callbacks = [
    tf.keras.callbacks.EarlyStopping(
        monitor='loss',
        patience=500,
        verbose=1,
        min_delta = 0,
        mode='auto',
        baseline=None,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join("/home/cbarker4/Documents/DataScience/StockTrader/Model", 'ckpt', "{epoch:02d}-{loss:.2f}.hdf5"),
        monitor='loss',
        verbose = 0,
        save_best_only = False,
        save_weights_only = False,
        mode = 'auto',
        save_freq='epoch',
        options=None,
        initial_value_threshold=None
    )   
]

In [7]:
class Custom_Ce_Loss(tf.keras.losses.Loss):
    def __init__(self):
        super().__init__()
    def call (self, y_true,y_pred):
        # if y_true[1]:
        #     if y_pred>y_true[0]:
        ans = tf.math.subtract(y_true[0],y_pred)
        ans = tf.math.square(ans)
        return tf.math.abs(ans)
        #     else:
        #         ans = tf.math.subtract(y_true[0],y_pred)
        #         ans =  tf.math.abs(ans)
        #         return tf.math.square(ans)
        # else:
        #     if y_pred<y_true[0]:
        #         ans = tf.math.subtract(y_true[0],y_pred)
        #         return tf.math.abs(ans)
        #     else:
        #         ans = tf.math.subtract(y_true[0],y_pred)
        #         ans =  tf.math.abs(ans)
        #         return tf.math.square(ans)

        

In [8]:
# loss_fun=["mean_absolute_error","KLD","MAE","MAPE","MSE","MSLE","binary_crossentropy","binary_focal_crossentropy","binary_crossentropy","binary_focal_crossentropy","categorical_accuracy","categorical crossentropy","deserialize","get","logcosh","hinge","kld","poisson","mse","mean_squared_logarithmic_error"]
# loss_fun = ["binary_crossentropy","binary_focal_crossentropy"]
# loss_fun = ["binary_crossentropy","binary_focal_crossentropy","categorical_accuracy","categorical crossentropy","deserialize"]
# loss_fun = ["mean_squared_logarithmic_error"]
loss_fun = ["MAE"]
best = {}
for losfun in loss_fun:
    # break

    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.LSTM(units=512,
                                return_sequences=True,
                                input_shape=(X_train.shape[1],1)))
    model.add(tf.keras.layers.LSTM(units=256,return_sequences=True))
    model.add(tf.keras.layers.LSTM(64))
    model.add(tf.keras.layers.Dense(32))
    # model.add(tf.keras.layers.Dense(64))
    # model.add(tf.keras.layers.Dense(32))

    model.add(tf.keras.layers.Dropout(0.5))
    
    model.add(tf.keras.layers.Dense(1))
    model.summary()


    

    # print(Y_train)
    
    

2023-04-29 16:00:30.427240: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-29 16:00:30.637098: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-29 16:00:30.637517: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-04-29 16:00:30.639771: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuil

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 100, 512)          1052672   
                                                                 
 lstm_1 (LSTM)               (None, 100, 256)          787456    
                                                                 
 lstm_2 (LSTM)               (None, 64)                82176     
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                                 
 dropout (Dropout)           (None, 32)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 1,924,417
Trainable params: 1,924,417
Non-

In [9]:
model.compile(optimizer='adam',
              loss='mean_squared_error')
model.fit(X_train,Y_train,epochs=1000, callbacks=callbacks)


Epoch 1/1000


2023-04-29 16:00:34.330214: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-04-29 16:00:35.959663: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-04-29 16:00:35.993439: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f65c0e72590 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-04-29 16:00:35.993471: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Ti, Compute Capability 8.6
2023-04-29 16:00:36.027215: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-04-29 16:00:36.328393: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime 

4/4 [==============================] - 5s 40ms/step - loss: 21257.6777
Epoch 2/1000
4/4 [==============================] - 0s 37ms/step - loss: 19890.5703
Epoch 3/1000
4/4 [==============================] - 0s 24ms/step - loss: 18981.4277
Epoch 4/1000
4/4 [==============================] - 0s 33ms/step - loss: 17885.5273
Epoch 5/1000
4/4 [==============================] - 0s 28ms/step - loss: 17231.0371
Epoch 6/1000
4/4 [==============================] - 0s 37ms/step - loss: 16562.0059
Epoch 7/1000
4/4 [==============================] - 0s 34ms/step - loss: 15803.1064
Epoch 8/1000
4/4 [==============================] - 0s 34ms/step - loss: 15199.3447
Epoch 9/1000
4/4 [==============================] - 0s 29ms/step - loss: 14593.7080
Epoch 10/1000
4/4 [==============================] - 0s 36ms/step - loss: 14051.8232
Epoch 11/1000
4/4 [==============================] - 0s 35ms/step - loss: 13696.3701
Epoch 12/1000
4/4 [==============================] - 0s 31ms/step - loss: 12957.2930
Ep

In [10]:
y_pred = model.predict(X_test)
print(y_pred)

2/2 [==============================] - 0s 5ms/step
[[143.96927]
 [143.97011]
 [143.96999]
 [143.97046]
 [143.97002]
 [143.96988]
 [143.96991]
 [143.96916]
 [143.96939]
 [143.97008]
 [143.96902]
 [143.97049]
 [143.96906]
 [143.97026]
 [143.96957]
 [143.97055]
 [143.97018]
 [143.9696 ]
 [143.96964]
 [143.96979]
 [143.96986]
 [143.97055]
 [143.96979]
 [143.96988]
 [143.97049]
 [143.97032]
 [143.97029]
 [143.96834]
 [143.96869]
 [143.96979]
 [143.9698 ]
 [143.96896]
 [143.97026]
 [143.9706 ]
 [143.97017]
 [143.97017]
 [143.96912]
 [143.96939]]


In [11]:

Xlast=[]
correct = 0
for val in X_test:
    Xlast.append(val[-1])


correct = 0
removed = 0
falsepos = 0
pos = 0
neg=0
falseneg =0
for i,val in enumerate(Y_test):
    if i < 10:
        print(Xlast[i],Y_test[i],y_pred[i])
    if Xlast[i]!=y_pred[i]:
        if Xlast[i] > val:

            if Xlast[i]>y_pred[i]:
                pos+=1
                correct+=1
            else :
                neg+=1
                falseneg+=1
        
        else:
            if Xlast[i]<y_pred[i]:
                correct+=1 
                neg+=1
            else:
                pos+=1
                falsepos+=1
    else:
        removed+=1

print('Correct predictions: ', correct)
print('Incorrect predictions: ', (len(y_pred)-removed) - correct)
print('Accuracy: ', correct / (len(y_pred)-removed))
print('False Pos:',falsepos)
print('Pos:',pos)
print('Pos Accuracy',(pos-falsepos)/pos)
print('Neg:',neg)
print('Neg Accuracy',(neg-falseneg)/neg)

135.21 137.87 [143.96927]
147.92 145.31 [143.97011]
142.48 142.45 [143.96999]
157.65 162.36 [143.97046]
148.03 147.81 [143.97002]
144.49 143.21 [143.96988]
143.21 134.51 [143.96991]
135.45 131.86 [143.96916]
134.87 149.7 [143.96939]
146.63 140.94 [143.97008]
Correct predictions:  24
Incorrect predictions:  14
Accuracy:  0.631578947368421
False Pos: 9
Pos: 17
Pos Accuracy 0.47058823529411764
Neg: 21
Neg Accuracy 0.7619047619047619
